In [4]:
from transformers import HubertPreTrainedModel, BertPreTrainedModel, BertModel, BertTokenizerFast
from transformers.models.hubert.modeling_hubert import HubertFeatureEncoder
from transformers.models.bert.modeling_bert import BertPooler

In [ ]:
class Hubert(HubertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        
        self.feature_projection = HubertFeatureEncoder(config)
        # self.feature_projection = HubertFeatureProjection(config)
        # self.encoder = HubertEncoder(config)
        
        self.post_init()
        
    def forward(
        self,
        input_values,
        attention_mask = None,
        mask_time_indices = None,
        output_attentions = None,
        output_hidden_states = None,
        return_dict = None,
    ):
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        extract_features = self.feature_extractor(input_values)
        extract_features = extract_features.transpose(1, 2)

        if attention_mask is not None:
            # compute reduced attention_mask corresponding to feature vectors
            attention_mask = self._get_feature_vector_attention_mask(extract_features.shape[1], attention_mask)

        hidden_states = self.feature_projection(extract_features)
        hidden_states = self._mask_hidden_states(hidden_states, mask_time_indices=mask_time_indices)

        encoder_outputs = self.encoder(
            hidden_states,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        hidden_states = encoder_outputs[0]

        if not return_dict:
            return (hidden_states,) + encoder_outputs[1:]

In [ ]:
projector = Hubert.from_pretrained('ntu-spml/distilhubert')

In [ ]:
class Pooler(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        
        self.pooler = BertPooler(config)
        
        self.post_init()
        
    def forward(self, hidden_states):
        return self.pooler(hidden_states)

In [ ]:
pooler = Pooler.from_pretrained('google/bert_uncased_L-2_H-128_A-2')

In [2]:
bert = BertModel.from_pretrained('google/bert_uncased_L-2_H-128_A-2')

Some weights of the model checkpoint at google/bert_uncased_L-2_H-128_A-2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
tokenizer = BertTokenizerFast.from_pretrained('google/bert_uncased_L-2_H-128_A-2')

Downloading: 100%|██████████| 226k/226k [00:00<00:00, 530kB/s] 


In [7]:
input_sentences = ['Hello, my dog is cute', 'Hello, my cat is cute']
bert_inputs = tokenizer(input_sentences, return_tensors='pt', max_length=32, padding=True, truncation=True)

In [9]:
bert_outputs = bert(input_ids=bert_inputs['input_ids'], attention_mask=bert_inputs['attention_mask'])

In [10]:
bert_outputs.pooler_output

tensor([[-1.0000,  0.0892, -0.9985,  0.9933, -0.9971,  0.7333, -0.9910, -0.8469,
          0.1403,  0.0398, -0.8121, -0.0226,  0.0342,  1.0000, -0.9129, -0.9487,
          0.9855,  0.0497, -0.8708,  0.9946,  0.8608,  0.0083,  0.9935,  0.9515,
         -1.0000, -0.0410, -0.9992,  0.9980,  0.9575,  0.1021,  0.0238,  0.0349,
         -0.9980, -0.2489,  0.9964,  0.9999, -0.8173, -0.0254, -0.0439, -0.9991,
          0.6942,  0.9822, -0.9986,  0.9773, -0.9999, -0.1172, -0.9998,  0.9988,
          0.9638,  0.9644,  0.8946, -0.9722,  0.0033,  0.9984,  0.9992,  0.9988,
         -0.9886, -0.8332,  0.3115,  0.2848, -0.0775,  0.0205,  0.6193,  0.9451,
         -0.4717, -1.0000,  0.3441,  0.2672,  0.9515,  0.6369,  0.9973, -0.0476,
         -0.9995,  0.0554,  0.5178, -0.9134, -0.1682,  0.1212, -0.9867,  0.0607,
         -0.2665, -0.0724, -0.9712, -0.9999,  0.9994, -0.9696,  0.5107,  0.3095,
         -0.3189,  0.8223, -0.6053,  0.9965, -0.9219,  0.9995,  0.7309,  0.6107,
         -0.9807, -0.8179, -